## How well can the level of corruption of a country in Europe be quantified? 

* What differences are there in actual corruption and perceived corruption? 

* Are there different forms of corruption prevalent in different countries in Europe? 

* What characteristics of a country predict the level of corruption? 

* What characteristics of a country predict an increase or decrease in the level of corruption?

• Corruption Perceptions Index (CPI) from Transparency International.
Data Set that shows preceived corruption of countries and rank them.

• World Bank Development Indicators (economic, social, and governance data).
Indicators could be used to look for correlation between them and the corruption score of countries

• European Social Survey (perception-related data).
• OECD Data on governance and public sector integrity.

In [55]:
import pandas as pd
import os

In [56]:
corruption_raw_data = pd.read_csv("../data/processed/CPI.csv")

In [57]:

corruption_raw_data.head()

,Economy ISO3,Economy Name,Indicator,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ALB,Albania,Corruption Perceptions Index Rank,113.0,116.0,110.00,88.00,83.00,91.00,99.00,106.00,104.00,110.00,101.00,98.00
1,ALB,Albania,Corruption Perceptions Index Standard Error,2.0,2.1,1.51,3.58,1.99,1.81,1.65,2.51,0.92,1.33,1.32,1.56
2,ALB,Albania,Corruption Perceptions Index Score,33.0,31.0,33.00,36.00,39.00,38.00,36.00,35.00,36.00,35.00,36.00,37.00
3,ALB,Albania,Corruption Perceptions Index Sources,7.0,7.0,7.00,7.00,7.00,8.00,8.00,8.00,8.00,8.00,8.00,7.00
4,AUT,Austria,Corruption Perceptions Index Rank,25.0,26.0,23.00,16.00,17.00,16.00,14.00,12.00,15.00,13.00,22.00,20.00


In [58]:
countries = pd.read_csv("../data/processed/europe_countries.csv")

In [59]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    48 non-null     object
 1   ISO3 Code  48 non-null     object
 2   ISO2 Code  48 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [60]:
iso3_europe_all = set(countries["ISO3 Code"])

In [61]:
len(iso3_europe_all)

48

In [62]:
iso3_europe_cpi = set(corruption_raw_data["Economy ISO3"])

In [63]:
len(iso3_europe_cpi)

42

In [64]:
iso3_europe_all-iso3_europe_cpi

{'AND', 'LIE', 'MCO', 'RKS', 'SMR', 'VAT'}

Countries missing: Andora, Liechtenstein, Kosovo, San Marino, Vatikan. I think we dont need this countries due to their size and small impact.

In [65]:
corruption_raw_data = corruption_raw_data[corruption_raw_data["Economy ISO3"].isin(iso3_europe_all)]
#corruption_raw_data = corruption_raw_data.drop(columns=["Indicator ID"])

# Transponing dataframe so that indicators are column-wise and years are in a single column
corruption_raw_data_melted = corruption_raw_data.melt(
    id_vars=["Economy ISO3", "Economy Name", "Indicator"],
    value_vars=[str(year) for year in range(2012, 2024)],
    var_name="year",
    value_name="Value"
)

# Pivot the melted DataFrame to make each indicator a column
corruption_transposed = corruption_raw_data_melted.pivot(
    index=["Economy ISO3", "Economy Name", "year"],  # Columns to keep as index
    columns="Indicator",  # Values to pivot into columns
    values="Value"  # Values to use in the new columns
).reset_index()

# Rename columns for clarity (optional)
corruption_transposed.columns.name = None
corruption_transposed.head()

,Economy ISO3,Economy Name,year,Corruption Perceptions Index Rank,Corruption Perceptions Index Score,Corruption Perceptions Index Sources,Corruption Perceptions Index Standard Error
0,ALB,Albania,2012,113.0,33.0,7.0,2.00
1,ALB,Albania,2013,116.0,31.0,7.0,2.10
2,ALB,Albania,2014,110.0,33.0,7.0,1.51
3,ALB,Albania,2015,88.0,36.0,7.0,3.58
4,ALB,Albania,2016,83.0,39.0,7.0,1.99


In [66]:
corruption_transposed.head()

,Economy ISO3,Economy Name,year,Corruption Perceptions Index Rank,Corruption Perceptions Index Score,Corruption Perceptions Index Sources,Corruption Perceptions Index Standard Error
0,ALB,Albania,2012,113.0,33.0,7.0,2.00
1,ALB,Albania,2013,116.0,31.0,7.0,2.10
2,ALB,Albania,2014,110.0,33.0,7.0,1.51
3,ALB,Albania,2015,88.0,36.0,7.0,3.58
4,ALB,Albania,2016,83.0,39.0,7.0,1.99


In [67]:
corruption_transposed["Economy ISO3"].nunique()

42

In [68]:
corruption_transposed.to_csv("../data/processed/CPI.csv", index=False, index_label=False)

In [69]:
cpi = pd.read_csv("../data/processed/CPI.csv")

In [70]:
cpi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 7 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Economy ISO3                                 504 non-null    object 
 1   Economy Name                                 504 non-null    object 
 2   year                                         504 non-null    int64  
 3   Corruption Perceptions Index Rank            491 non-null    float64
 4   Corruption Perceptions Index Score           504 non-null    float64
 5   Corruption Perceptions Index Sources         504 non-null    float64
 6   Corruption Perceptions Index Standard Error  504 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 27.7+ KB
